# 1. titanic 예측
* titanic_cleaning.csv 파일을 불러와서 진행하시오.
* PassengerId는 순차적인 번호이므로 제외하고 진행하시오
* train, test 80:20 으로 진행하시오
* 여러가지 모델을 생성 후 최적의 모델을 이용하시오
* 최적의 모델을 찾고 자신이 탑승했을 경우 인적사항을 넣고 결과를 예측하시오
* 컬럼
    - PassengerId : 고객 번호
    - Survived : 0이면 사망, 1이면 생존
    - Pclass : 티켓 등급으로 1, 2, 3
    - Sex : 성별, 0이면 남성, 1이면 여성
    - Age : 나이
    - SibSp : 형제, 자매, 배우자의 합
    - Parch : 부모, 자식의 합
    - Fare : 요금

In [1]:
import pandas as pd
df = pd.read_csv('../data_set/4.분류/titanic_cleaning.csv')
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,0,22.0,1,0,7.2500
1,2,1,1,1,38.0,1,0,71.2833
2,3,1,3,1,26.0,0,0,7.9250
3,4,1,1,1,35.0,1,0,53.1000
4,5,0,3,0,35.0,0,0,8.0500


# 2. feature, label 분리 

In [2]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare'],
      dtype='object')

In [3]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
label = 'Survived'
X = df[features]
y = df[label]

In [ ]:
# 훈련, 테스트 셋 분리 

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[features], df[label], test_size=0.2)
print("총 개수 : ", X.shape, y.shape)
print("train 개수 : ", X_train.shape, y_train.shape)
print("test 개수 : ", X_test.shape, y_test.shape)

총 개수 :  (891, 6) (891,)
train 개수 :  (712, 6) (712,)
test 개수 :  (179, 6) (179,)


In [5]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier()
kn.fit(X_train, y_train)
kn.score(X_test, y_test)

0.659217877094972

In [6]:
import sklearn.svm as svm
svm_linear = svm.SVC(kernel="linear")
svm_linear.fit(X_train, y_train)
svm_linear.score(X_test, y_test)

0.7988826815642458

In [7]:
svm_rbf = svm.SVC(kernel="rbf")
svm_rbf.fit(X_train, y_train)
svm_rbf.score(X_test, y_test)

0.6312849162011173

In [8]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_clf.score(X_test, y_test)

0.7486033519553073

In [9]:
svm_rbf = svm.SVC(kernel="rbf", probability=True)
svm_rbf.fit(X_train, y_train)

kn = KNeighborsClassifier()
kn.fit(X_train, y_train)

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [10]:
from sklearn.ensemble import VotingClassifier
vo = VotingClassifier(estimators=[ ('svc', svm_rbf), ('knn', kn), ('DecisionTree', dt_clf)] , voting='soft')
vo.fit(X_train, y_train)

print("svm_rbf : ", svm_rbf.score(X_test, y_test))
print("kn : ", kn.score(X_test, y_test))
print("dt_clf : ", dt_clf.score(X_test, y_test))
print("vo : ", vo.score(X_test, y_test))

svm_rbf :  0.6312849162011173
kn :  0.659217877094972
dt_clf :  0.7597765363128491
vo :  0.7374301675977654


In [11]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.770949720670391

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
gbc.score(X_test, y_test)

0.7932960893854749

# 3. 생존, 죽음 여부 확인

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
# ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
test = [[1, 0, 35, 1, 1, 10]]
result = rfc.predict(test)
result

array([0], dtype=int64)

In [16]:
if (result[0] == 1):
    print("생존")
else :
    print("T^T")

T^T


# 4. 평가 지표로 확인하기

In [17]:
pred = rfc.predict(X_test)

In [ ]:
# 예측 정확도 평가 (Accuracy를 통한 성능평가)

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, pred)   # 정확도
precision = precision_score(y_test, pred) # 정밀도
recall = recall_score(y_test, pred)       # 재현율
f1 = f1_score(y_test, pred)               # f1 score

print("정확도 : ", accuracy)
print("정밀도 : ", precision)
print("재현율 : ", recall)
print("f1 score : ", f1)

정확도 :  0.770949720670391
정밀도 :  0.7931034482758621
재현율 :  0.6133333333333333
f1 score :  0.6917293233082706


# 5. 결론
* 정확도 :  맞춘 확률이라고 보면 된다.
* 재현율(recall)
    - 실제로 1(생존)에 대해 예측 1(생존)일 경우에 영향을 미친다.
    - 실제 0(사망)에 대해 예측 1(생존)을 잘못 맞춘 경우에는 영향을 미치지 않는다.
    - 그럼 지금과 같은 상황에서 실제 사망한 사람을 살았다고 예측한거에 대해 영향이 없으면 문제가 생길 수 있을 것 같다.
* 정밀도(precision)
    - 예측 1(생존)에 대해 실제 1(생존)일 경우에 영향을 미친다.
    - 실제 1(생존)을 예측 0(사망)으로 예측한 경우에는 척도에 영향을 미치지 않는다.
    - 그럼 지금과 같은 상황에서 실제 1(생존)이라는 값을 0(사망)으로 잘못 예측하게 된다면, 재현율보다는 정밀도의 척도가 옳을 것이다.
    - 사망했다고 판단하였으나 살아서 돌아온다면 뭐라고 할 사람은 아무도 없을 것이다.
    - 하지만 반대로 생존했다고 판단했지만 실제로는 사망했다고 한다면, 이전의 판단보다는 파급 효과가 클 것으로 보인다. 